# Questions and Answers using Llama 2

Model: Llama 2 7b-chat

Data: Handbook test data set

In [6]:
from reginald.models.setup_llm import setup_llm

from reginald.models.models.llama_index import (
    setup_settings,
    LlamaIndexLlamaCPP,
    set_global_tokenizer,
    compute_default_chunk_size,
)


In [7]:
from llama_index.core.evaluation import DatasetGenerator, RelevancyEvaluator
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Response
from llama_index.llms.openai import OpenAI
from llama_index.llms.ollama import Ollama

from llama_index.core import (
    StorageContext,
    load_index_from_storage,
)

from transformers import AutoTokenizer
import nest_asyncio

nest_asyncio.apply()

In [8]:
# Use ollama
ollama_llm = Ollama(model="llama2:7b-chat", request_timeout=60.0)

In [9]:
chunk_size = compute_default_chunk_size(
    max_input_size=4096, k=3
)  # calculate chunk size

In [10]:
tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf"
).encode  # load tokenizer
set_global_tokenizer(tokenizer)

In [11]:
settings = setup_settings(
    llm=ollama_llm,
    max_input_size=4096,
    num_output=512,
    chunk_overlap_ratio=0.1,
    chunk_size=chunk_size,
    k=3,
    tokenizer=tokenizer,
)  # these are settings for the storage context

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Load in the Data

In [12]:
# To load in the handbook

storage_context = StorageContext.from_defaults(
    persist_dir="../../data/llama_index_indices/handbook/"
)

vector_index = load_index_from_storage(
    storage_context=storage_context,
    settings=settings,
)  # load the data index from storage

In [13]:
# To load in the example Paul Graham data
# reader = SimpleDirectoryReader("./data/paul_graham/")
# documents = reader.load_data()
# vector_index = VectorStoreIndex.from_documents(documents)

In [14]:
dataset_generator = DatasetGenerator.from_documents(
    vector_index.docstore.docs.values(),
    num_questions_per_chunk=1,
    show_progress=True,
    llm=ollama_llm
)

Parsing nodes:   0%|          | 0/39 [00:00<?, ?it/s]

/Users/kgoldmann/Documents/Projects/reginald/.venv/lib/python3.11/site-packages/llama_index/core/evaluation/dataset_generation.py:213: DeprecationWarning: Call to deprecated class DatasetGenerator. (Deprecated in favor of `RagDatasetGenerator` which should be used instead.)
  return cls(


## Generate Questions

In [15]:
eval_questions = dataset_generator.generate_questions_from_nodes(
    5
)  # generate questions from the documents

100%|██████████| 5/5 [00:37<00:00,  7.55s/it]
/Users/kgoldmann/Documents/Projects/reginald/.venv/lib/python3.11/site-packages/llama_index/core/evaluation/dataset_generation.py:310: DeprecationWarning: Call to deprecated class QueryResponseDataset. (Deprecated in favor of `LabelledRagDataset` which should be used instead.)
  return QueryResponseDataset(queries=queries, responses=responses_dict)


In [16]:
eval_questions

['Of course! Here are 5 questions based on the provided context information:',
 'What is the purpose of the "Contributing" section in the REG Handbook?',
 'Who is the intended audience for the guide in the "Contributing" section?',
 'How does the guide aim to make contributing to the handbook user-friendly?',
 'What is the main focus of the guide in terms of providing links for further reading?']

## Answer the Questions

In [17]:
evaluator_gpt4 = RelevancyEvaluator(llm=ollama_llm)

In [18]:
# for every value in eval_questions, get the response from the llm
all_dfs = []

for i in range(len(eval_questions)):
    query_engine = vector_index.as_query_engine()
    response_vector = query_engine.query(eval_questions[i])
    eval_result = evaluator_gpt4.evaluate_response(
        query=eval_questions[i], response=response_vector
    )

    # append to the all_dfs list
    all_dfs.append({'eval_result': eval_result,
                    'question': eval_questions[i],
                    'response': response_vector})


In [21]:
# extract the different elements
questions = [x['question'] for x in all_dfs]
response = [x['response'] for x in all_dfs]
sources = [x['response'].source_nodes for x in all_dfs]
#sources = [x[0].node.get_content() for x in sources]

# for the each sources, get the content as one string deliminated by ;
sources = [";".join([x.get_content() for x in y]) for y in sources]

match = [x['eval_result'].passing for x in all_dfs] # does the response match the source

In [25]:
# extract the questions
combined_df = pd.DataFrame({'questions': questions, 'response': response, 'sources': sources, 'match': match})
combined_df.to_csv("data/handbook_QandA.csv", index=False)

In [26]:
combined_df_out = combined_df.style.set_properties(
        **{
            "inline-size": "600px",
            "overflow-wrap": "break-word",
        }
    )
display(combined_df_out)